In [ ]:

import gradio as gr
import requests

# Function to call the FastAPI endpoint and get drug interactions
def get_interactions(drug_names, tpuid_ids):
    url = "http://quanta2.manageai.co.th:7777/interactions/"
    # Prepare the data to send to FastAPI
    data = {
        "input_text": drug_names.split(","),
        "tpu_id": [int(x) for x in tpuid_ids.split(",")]
    }
    response = requests.post(url, json=data)

    if response.status_code == 200:
        interactions_data = response.json().get("interactions", [])
        interaction_pairs = []
        # Prepare a list of drug pairs to show in the dropdown
        for interaction in interactions_data:
            pair_name = f"{interaction['drug_name1']} <-> {interaction['drug_name2']}"
            interaction_pairs.append(pair_name)  # Just names for dropdown
        return interaction_pairs, interactions_data  # Return pairs and full interaction data
    else:
        print(f"Error fetching data: {response.status_code}")
        return [], []



# Define sample input sets for convenience
def set_sample_input(sample_set):
    samples = {
        "Set 1": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML,ASPIRIN 300 MG TAB,BEFARIN 3", "262926, 519545,226243,108159"),
        "Set 2": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML", "262926, 519545"),
        "Set 3": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML,ASPIRIN 300 MG TAB,BEFARIN 3", "262926, 519545, 226243, 108159"),
        "Set 4": ("AAAA,BBBB", "0000000, 0000000")  # No interactions expected
    }
    return samples.get(sample_set, ("", ""))

# Gradio UI Setup
with gr.Blocks() as demo:
    gr.Markdown("## 💊 Drug Interaction Checker")

    # Dropdown for selecting sample set
    sample_set_dropdown = gr.Dropdown(
        choices=["Select Sample Set", "Set 1", "Set 2", "Set 3", "Set 4"],
        label="Select Test Sample",
        interactive=True
    )

    # Inputs for drug names and TPUIDs
    with gr.Row():
        drug_input = gr.Textbox(label="Enter Drug Names (comma-separated)")
        tpu_input = gr.Textbox(label="Enter TPUIDs (comma-separated)")

    submit_btn = gr.Button("Check Interactions")

    # Dropdown for displaying interactions
    interaction_output = gr.Dropdown(
        label="Select Drug Interaction",
        choices=[],  # Start with an empty list
        interactive=True
    )

    # Textboxes for displaying details of the selected interaction
    with gr.Row():
        significance_box = gr.Textbox(label="Significance", interactive=False)
        onset_box = gr.Textbox(label="Onset", interactive=False)
        severity_box = gr.Textbox(label="Severity", interactive=False)
        documentation_box = gr.Textbox(label="Documentation", interactive=False)

    # with gr.Accordion("Effect", open=True):
    #     effect_box = gr.Textbox(label="Effect", interactive=False, lines=1.5)

    with gr.Accordion("Mechanism", open=True):
        mechanism_box = gr.Textbox(label="Mechanism", interactive=False, lines=1.5)

    with gr.Accordion("Management", open=True):
        management_box = gr.Textbox(label="Management", interactive=False, lines=1.5)

    with gr.Accordion("Discussion", open=True):
        discussion_box = gr.Textbox(label="Discussion", interactive=False, lines=1.5)

    footer = gr.Markdown("")

    # Link sample set selection to the inputs
    sample_set_dropdown.change(
        set_sample_input,
        inputs=[sample_set_dropdown],
        outputs=[drug_input, tpu_input]
    )

        # Handle the submit button to fetch interactions
    def on_submit(drug_names, tpuid_ids):
        interactions, interaction_data = get_interactions(drug_names, tpuid_ids)
        print(f"Fetched interactions: {interactions}")  # Log fetched interactions
        print(f"Interaction Data: {interaction_data}")  # Log the entire interaction data for inspection

        if not interactions:
            return [], [], []  # Return empty dropdown if no interactions found
        return gr.update(choices=interactions), gr.update(value=interaction_data), gr.State(interaction_data)

    # Add an error output (gr.Textbox) for error messages
    error_message_box = gr.Textbox(label="Error", interactive=False, visible=False)

    submit_btn.click(
        on_submit,
        inputs=[drug_input, tpu_input],
        outputs=[interaction_output, error_message_box, gr.State()]
    )

    def store_detail(interaction_data):
        # If there's no interaction data, return empty fields
        if not interaction_data:
            print("No interaction data available.")  # Log when no data exists
            return [["", "", "", "", "", "", ""]]  # Return an empty set of values

        all_details = []
        # Iterate over all interaction data and collect the details
        for details in interaction_data:
            print(f"Found details: {details}")  # Log the found details

            # Collect the details for each interaction, defaulting to empty string if not available
            interaction_details = [
                # details.get("interaction_detail",""),
                details.get("Significance", ""),
                details.get("Onset", ""),
                details.get("Severity", ""),
                details.get("Documentation", ""),
                details.get("mechanism", ""),
                details.get("management", ""),
                details.get("discussion", ""),
            ]

            all_details.append(interaction_details)
        return all_details

    def on_interaction_select(selected_interaction, drug_input, tpu_input):
        print(f"Selected interaction: {selected_interaction}")  # Log the selected interaction
        interactions, interaction_detail = get_interactions(drug_input, tpu_input)
        
        for idx, interaction in enumerate(interactions):
            if selected_interaction == interaction:
                print(f"Found matching interaction: {selected_interaction}")  # Log when a match is found
    
                # Extract the corresponding interaction_detail for the matched interaction
                matching_interaction_detail = interaction_detail[idx]
                print(f"Interaction detail: {matching_interaction_detail}")  # Log the corresponding interaction details
    
                # Store the matching details using store_detail function
                my_store = store_detail([matching_interaction_detail])  # จะได้ list ซ้อน [[...]]
                print(tuple(my_store[0]))
                # ต้องแยกค่าออกมาเป็น 8 ตัว ไม่ใช่ list ซ้อน
                return tuple(my_store[0])  # ใช้ tuple เพื่อให้แยกค่าเป็น 8 ตัว


    # Update the interaction details on selection
    interaction_output.select(
        on_interaction_select,
        inputs=[interaction_output, drug_input, tpu_input],
        outputs=[
            significance_box, onset_box, severity_box, documentation_box, mechanism_box, management_box, discussion_box
        ]
    )

# Launch the Gradio interface
demo.launch(debug=True)


     